In [ ]:
# %load_ext cudf.pandas
import pandas as pd
import json

pasta = 'stz/'

In [ ]:
dados_input = json.load(open(f"{pasta}meta_dados.json", "r", encoding="cp1252"))
dados = dados_input["dados"]
metadados = dados_input["metadados"]

dados.keys(), metadados.keys()

In [ ]:
nos = pd.DataFrame(dados["criacaoDeNos"])
# criar coluna id, que corresponde ao nivel do no e um numero sequencial iniciando em 1 para cada nivel
nos["idn"] = nos.groupby("nivel").cumcount() + 1
nos["idn"] = nos["nivel"] + "_" + nos["idn"].astype(str)
replace_nos = dict(zip(nos["no"], nos["idn"]))
nos.to_csv(f"{pasta}nos.csv", index=False)
nos.drop(columns=["no"]).to_csv(f"{pasta}nos_anonim.csv", index=False)
nos

In [ ]:
skus = pd.DataFrame(dados["skus"])
skus["ids"] = "sku_" + skus.index.astype(str)
skus.to_csv(f"{pasta}skus.csv", index=False)
replace_skus = dict(zip(skus["listaDeSkus"], skus["ids"]))
skus.drop(columns=["listaDeSkus"]).to_csv(f"{pasta}skus_anonim.csv", index=False)
skus

In [ ]:
clusters = pd.DataFrame(dados["clustersSkuAtribuicao"])
if not clusters.empty:
    clusters = clusters[["sku", "categoriaDeSku"]].rename(columns={"categoriaDeSku": "cluster"})
    clusters["sku"] = clusters["sku"].replace(replace_skus)
else:
    clusters = pd.DataFrame(columns=["sku", "cluster"])

clusters

In [ ]:
fretes  = list()

frete = pd.DataFrame(dados["freteBase"])
if not frete.empty:
    frete["origem"] = frete["origem"].replace(replace_nos)
    frete["destino"] = frete["destino"].replace(replace_nos)

    frete = frete.merge(skus.rename(columns={"ids": "sku"}), how="cross")
    frete.to_csv(f"{pasta}frete_base.csv", index=False)
    print(frete.head())
    fretes.append(frete[["origem", "destino", "sku", "perfil", "custo"]])

frete = pd.DataFrame(dados["fretePorCluster"])
if not frete.empty:
    clusters = pd.DataFrame(dados["clustersSkuAtribuicao"])[["sku", "categoriaDeSku"]].rename(columns={"categoriaDeSku": "cluster"})
    clusters["sku"] = clusters["sku"].replace(replace_skus)
    frete["origem"] = frete["origem"].replace(replace_nos)
    frete["destino"] = frete["destino"].replace(replace_nos)

    frete = frete.merge(clusters, on="cluster", how="left")

    frete.to_csv(f"{pasta}frete_cluster.csv", index=False)
    print(frete.head())
    fretes.append(frete[["origem", "destino", "sku", "perfil", "custo"]])

frete = pd.DataFrame(dados["fretePorSku"])
if not frete.empty:
    frete["origem"] = frete["origem"].replace(replace_nos)
    frete["destino"] = frete["destino"].replace(replace_nos)
    frete["sku"] = frete["sku"].replace(replace_skus)
    frete.to_csv(f"{pasta}frete_sku.csv", index=False)
    print(frete.head())
    fretes.append(frete[["origem", "destino", "sku", "perfil", "custo"]])

if len(fretes) > 1:
    frete_total = pd.concat(fretes).groupby(["origem", "destino", "sku", "perfil"]).sum().reset_index()
else:
    frete_total = fretes[0]
frete_total.to_csv(f"{pasta}frete.csv", index=False)
print(frete_total.head())

In [ ]:
km_rota = pd.DataFrame(dados["kmDaRota"])
km_rota["origem"] = km_rota["origem"].replace(replace_nos)
km_rota["destino"] = km_rota["destino"].replace(replace_nos)
km_rota.to_csv(f"{pasta}km_rota.csv", index=False)
km_rota

In [ ]:
baseline = pd.DataFrame(dados["baseline"])
baseline["origem"] = baseline["origem"].replace(replace_nos)
baseline["destino"] = baseline["destino"].replace(replace_nos)
baseline["sku"] = baseline["sku"].replace(replace_skus)
baseline.to_csv(f"{pasta}baseline.csv", index=False)
baseline

In [ ]:
valor_mercadoria = pd.DataFrame(dados["porRota"])
valor_mercadoria["origem"] = valor_mercadoria["origem"].replace(replace_nos)
valor_mercadoria["destino"] = valor_mercadoria["destino"].replace(replace_nos)
valor_mercadoria["sku"] = valor_mercadoria["sku"].replace(replace_skus)
valor_mercadoria.to_csv(f"{pasta}valor_mercadoria.csv", index=False)
valor_mercadoria

In [ ]:
demanda = pd.DataFrame(dados["demandaPorSku"])
demanda["sku"] = demanda["sku"].replace(replace_skus)
demanda["no"] = demanda["no"].replace(replace_nos)
demanda.to_csv(f"{pasta}demanda.csv", index=False)
demanda

In [ ]:
custo_nos = pd.DataFrame(dados["no"])
custo_nos["no"] = custo_nos["no"].replace(replace_nos)
custo_nos.to_csv(f"{pasta}custo_nos.csv", index=False)
custo_nos